A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from mingpt.utils import set_seed
from mingpt.model import GPT
from mingpt.trainer import Trainer

import pickle
import random

set_seed(3407)

In [2]:
# class SortDataset(Dataset):
#     """ 
#     Dataset for the Sort problem. E.g. for problem length 6:
#     Input: 0 0 2 1 0 1 -> Output: 0 0 0 1 1 2
#     Which will feed into the transformer concatenated as:
#     input:  0 0 2 1 0 1 0 0 0 1 1
#     output: I I I I I 0 0 0 1 1 2
#     where I is "ignore", as the transformer is reading the input sequence
#     """

#     def __init__(self, split, length=6, num_digits=3):
#         assert split in {'train', 'test'}
#         self.split = split
#         self.length = length
#         self.num_digits = num_digits
    
#     def __len__(self):
#         return 10000 # ...
    
#     def get_vocab_size(self):
#         return self.num_digits
    
#     def get_block_size(self):
#         # the length of the sequence that will feed into transformer, 
#         # containing concatenated input and the output, but -1 because
#         # the transformer starts making predictions at the last input element
#         return self.length * 2 - 1

#     def __getitem__(self, idx):
        
#         # use rejection sampling to generate an input example from the desired split
#         while True:
#             # generate some random integers
#             inp = torch.randint(self.num_digits, size=(self.length,), dtype=torch.long)
#             # half of the time let's try to boost the number of examples that 
#             # have a large number of repeats, as this is what the model seems to struggle
#             # with later in training, and they are kind of rate
#             if torch.rand(1).item() < 0.5:
#                 if inp.unique().nelement() > self.length // 2:
#                     # too many unqiue digits, re-sample
#                     continue
#             # figure out if this generated example is train or test based on its hash
#             h = hash(pickle.dumps(inp.tolist()))
#             inp_split = 'test' if h % 4 == 0 else 'train' # designate 25% of examples as test
#             if inp_split == self.split:
#                 break # ok
        
#         # solve the task: i.e. sort
#         sol = torch.sort(inp)[0]

#         # concatenate the problem specification and the solution
#         cat = torch.cat((inp, sol), dim=0)

#         # the inputs to the transformer will be the offset sequence
#         x = cat[:-1].clone()
#         y = cat[1:].clone()
#         # we only want to predict at output locations, mask out the loss at the input locations
#         y[:self.length-1] = -1
#         return x, y


In [2]:
def pad_with_zeros_str_mult(s, length):
    return (2 * length - len(s)) * '0' + s if (2 * length - len(s)) >0 else s

def random_add_instance(length):
    a = [random.randint(0,9) for i in range(length)]
    b = [random.randint(0,9) for i in range(length)]
    val_a = int(''.join(str(d) for d in a))
    val_b = int(''.join(str(d) for d in b))
    val_c = val_a + val_b
    str_c = str(val_c)
    str_c = (length + 1 - len(str_c)) * '0' + str_c
    return a + b + [int(d) for d in str_c]

def random_multiply_instance(length):
    a = [random.randint(0,9) for i in range(length)]
    b = [random.randint(0,9) for i in range(length)]
    val_a = int(''.join(str(d) for d in a))
    val_b = int(''.join(str(d) for d in b))
    val_c = val_a * val_b
    str_c = str(val_c)
    str_c = (2 * length - len(str_c)) * '0' + str_c if (2 * length - len(str_c)) >0 else str_c
    return a + b + [int(d) for d in str_c]

def random_multiply_long_instance(length):
    a = [random.randint(0,9) for i in range(length)]
    b = [random.randint(0,9) for i in range(length)]
    # a = [1, 2 , 3]
    # b = [4, 5, 6]
    val_a = int(''.join(str(d) for d in a))
    val_b = int(''.join(str(d) for d in b))
    elems = []
    ten_pow = 1
    for d in b[::-1]:
        e_val = val_a * d * ten_pow
        e_str = pad_with_zeros_str_mult(str(e_val), length)
        elems.append(e_str)
        ten_pow *= 10

    val_c = val_a * val_b
    str_c = str(val_c)
    str_c = (2 * length - len(str_c)) * '0' + str_c if (2 * length - len(str_c)) >0 else str_c
    return a + b + [int(d) for d in "".join(elems)] + [int(d) for d in str_c]


for i in range(10):
#     # print (random_add_instance(3))
    print(random_multiply_long_instance(3))
#     print(random_multiply_instance(3))



[0, 6, 2, 5, 3, 6, 0, 0, 0, 3, 7, 2, 0, 0, 1, 8, 6, 0, 0, 3, 1, 0, 0, 0, 0, 3, 3, 2, 3, 2]
[4, 9, 5, 3, 7, 8, 0, 0, 3, 9, 6, 0, 0, 3, 4, 6, 5, 0, 1, 4, 8, 5, 0, 0, 1, 8, 7, 1, 1, 0]
[6, 4, 4, 3, 1, 4, 0, 0, 2, 5, 7, 6, 0, 0, 6, 4, 4, 0, 1, 9, 3, 2, 0, 0, 2, 0, 2, 2, 1, 6]
[3, 5, 9, 1, 1, 9, 0, 0, 3, 2, 3, 1, 0, 0, 3, 5, 9, 0, 0, 3, 5, 9, 0, 0, 0, 4, 2, 7, 2, 1]
[3, 5, 0, 1, 4, 8, 0, 0, 2, 8, 0, 0, 0, 1, 4, 0, 0, 0, 0, 3, 5, 0, 0, 0, 0, 5, 1, 8, 0, 0]
[0, 0, 2, 9, 3, 7, 0, 0, 0, 0, 1, 4, 0, 0, 0, 0, 6, 0, 0, 0, 1, 8, 0, 0, 0, 0, 1, 8, 7, 4]
[8, 4, 8, 2, 3, 7, 0, 0, 5, 9, 3, 6, 0, 2, 5, 4, 4, 0, 1, 6, 9, 6, 0, 0, 2, 0, 0, 9, 7, 6]
[9, 9, 9, 7, 0, 6, 0, 0, 5, 9, 9, 4, 0, 0, 0, 0, 0, 0, 6, 9, 9, 3, 0, 0, 7, 0, 5, 2, 9, 4]
[7, 4, 5, 1, 3, 6, 0, 0, 4, 4, 7, 0, 0, 2, 2, 3, 5, 0, 0, 7, 4, 5, 0, 0, 1, 0, 1, 3, 2, 0]
[1, 0, 1, 0, 4, 2, 0, 0, 0, 2, 0, 2, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 4, 2]


In [4]:
class AddDataset(Dataset):
    """ 
    Dataset for the Add problem. E.g. for problem length 3:
    12 + 333 = 345
    Input: 0 1 2 3 3 3 -> Output: 0 3 4 5
    Which will feed into the transformer concatenated as:
    input:  0 1 2 3 3 3 0 3 4
    output: I I I I I 0 3 4 5
    where I is "ignore", as the transformer is reading the input sequence
    """

    def __init__(self, split, length=3):
        assert split in {'train', 'test'}
        self.split = split
        self.length = length
    
    def __len__(self):
        return 10000 # ...
    
    def get_vocab_size(self):
        return 10
    
    def get_block_size(self):
        # the length of the sequence that will feed into transformer, 
        # containing concatenated input and the output, but -1 because
        # the transformer starts making predictions at the last input element
        return 3 * self.length + 1 - 1

    def __getitem__(self, idx):
        while True:
            rai = random_add_instance(self.length)
            h = hash(str(rai[:2*self.length]))
            
            inp_split = 'test' if h % 4 == 0 else 'train' # designate 25% of examples as test
            if inp_split == self.split:
                break # ok
        
        x = torch.tensor(rai[:-1], dtype=torch.long)
        y = torch.tensor(rai[1:], dtype=torch.long)
        
        # we only want to predict at output locations, mask out the loss at the input locations
        y[:2*self.length-1] = -1
        return x, y

In [3]:
class MultiplyDataset(Dataset):
    """ 
    Dataset for the Multiply problem. E.g. for problem length 3:
    12 * 333 = 345
    Input: 0 1 2 3 3 3 -> Output: 0 0 3 9 9 6
    Which will feed into the transformer concatenated as:
    input:  0 1 2 3 3 3 0 0 3 9 9
    output: I I I I I 0 0 3 9 9 6
    where I is "ignore", as the transformer is reading the input sequence
    """

    def __init__(self, split, length=3):
        assert split in {'train', 'test'}
        self.split = split
        self.length = length
    
    def __len__(self):
        return 10000 # ...
    
    def get_vocab_size(self):
        return 10
    
    def get_block_size(self):
        # the length of the sequence that will feed into transformer, 
        # containing concatenated input and the output, but -1 because
        # the transformer starts making predictions at the last input element
        return 4 * self.length - 1

    def __getitem__(self, idx):
        while True:
            rai = random_multiply_instance(self.length)
            h = hash(str(rai[:2*self.length]))
            
            inp_split = 'test' if h % 4 == 0 else 'train' # designate 25% of examples as test
            if inp_split == self.split:
                break # ok
        
        x = torch.tensor(rai[:-1], dtype=torch.long)
        y = torch.tensor(rai[1:], dtype=torch.long)
        
        # we only want to predict at output locations, mask out the loss at the input locations
        y[:2*self.length-1] = -1
        return x, y

In [4]:
class MultiplyLongDataset(Dataset):
    """ 
    Dataset for the Multiply problem. E.g. for problem length 3:
    12 * 333 = 345
    Input: 0 1 2 3 3 3 -> Output: 0 0 3 9 9 6
    Which will feed into the transformer concatenated as:
    input:  0 1 2 3 3 3 0 0 3 9 9
    output: I I I I I 0 0 3 9 9 6
    where I is "ignore", as the transformer is reading the input sequence
    """

    def __init__(self, split, length=3):
        assert split in {'train', 'test'}
        self.split = split
        self.length = length
    
    def __len__(self):
        return 10000 # ...
    
    def get_vocab_size(self):
        return 10
    
    def get_block_size(self):
        # the length of the sequence that will feed into transformer, 
        # containing concatenated input and the output, but -1 because
        # the transformer starts making predictions at the last input element
        return 2*self.length + self.length*2*self.length + 2*self.length - 1

    def __getitem__(self, idx):
        while True:
            rai = random_multiply_long_instance(self.length)
            h = hash(str(rai[:2*self.length]))
            
            inp_split = 'test' if h % 4 == 0 else 'train' # designate 25% of examples as test
            if inp_split == self.split:
                break # ok
        
        x = torch.tensor(rai[:-1], dtype=torch.long)
        y = torch.tensor(rai[1:], dtype=torch.long)
        
        # we only want to predict at output locations, mask out the loss at the input locations
        y[:2*self.length + self.length*2*self.length - 1] = -1
        return x, y

In [7]:
# print an example instance of the dataset
train_dataset = AddDataset('train')
test_dataset = AddDataset('test')
x, y = train_dataset[0]

print (x)
for a, b in zip(x,y):
    print(int(a),int(b))

tensor([1, 9, 0, 0, 1, 7, 0, 2, 0])
1 -1
9 -1
0 -1
0 -1
1 -1
7 0
0 2
2 0
0 7


In [15]:
train_dataset_m = MultiplyDataset('train')
test_dataset_m = MultiplyDataset('test')
x_m, y_m = train_dataset_m[0]

print (x_m, y_m[-1].item())
for a_m, b_m in zip(x_m,y_m):
    print(int(a_m),int(b_m))

tensor([0, 8, 1, 3, 0, 8, 0, 2, 4, 9, 4]) 8
0 -1
8 -1
1 -1
3 -1
0 -1
8 0
0 2
2 4
4 9
9 4
4 8


In [5]:
train_dataset_ml = MultiplyLongDataset('train')
test_dataset_ml = MultiplyLongDataset('test')
x_ml, y_ml = train_dataset_ml[0]

print (x_ml, y_ml[-1].item())
for a_ml, b_ml in zip(x_ml,y_ml):
    print(int(a_ml),int(b_ml))

tensor([9, 2, 5, 1, 1, 1, 0, 0, 0, 9, 2, 5, 0, 0, 9, 2, 5, 0, 0, 9, 2, 5, 0, 0,
        1, 0, 2, 6, 7]) 5
9 -1
2 -1
5 -1
1 -1
1 -1
1 -1
0 -1
0 -1
0 -1
9 -1
2 -1
5 -1
0 -1
0 -1
9 -1
2 -1
5 -1
0 -1
0 -1
9 -1
2 -1
5 -1
0 -1
0 1
1 0
0 2
2 6
6 7
7 5


In [10]:
# create a GPT instance


# model_config = GPT.get_default_config()
# model_config.model_type = 'gpt-micro'
# model_config.model_type = 'gpt-nano'

# model_config.vocab_size = train_dataset.get_vocab_size()
# model_config.block_size = train_dataset.get_block_size()
# model = GPT(model_config)

In [11]:
model_config_m = GPT.get_default_config()
model_config_m.model_type = 'gpt-micro'
# model_config_m.model_type = 'gpt-nano'

model_config_m.vocab_size = train_dataset_m.get_vocab_size()
model_config_m.block_size = train_dataset_m.get_block_size()
model_m = GPT(model_config_m)

number of parameters: 0.80M


In [6]:
model_config_ml = GPT.get_default_config()
model_config_ml.model_type = 'gpt-micro'
# model_config_m.model_type = 'gpt-nano'

model_config_ml.vocab_size = train_dataset_ml.get_vocab_size()
model_config_ml.block_size = train_dataset_ml.get_block_size()
model_ml = GPT(model_config_ml)

number of parameters: 0.80M


In [14]:
# print (model_config.n_head, model_config.n_layer, model_config.n_embd)

In [15]:
# create a Trainer object

# train_config = Trainer.get_default_config()
# train_config.learning_rate = 5e-4 # the model we're using is so small that we can go a bit faster
# train_config.max_iters = 5000
# train_config.num_workers = 0
# trainer = Trainer(train_config, model, train_dataset)

In [16]:
train_config_m = Trainer.get_default_config()
train_config_m.learning_rate = 5e-3 # the model we're using is so small that we can go a bit faster
train_config_m.max_iters = 10000
train_config_m.num_workers = 0

# train_config_m.learning_rate = 1e-3  # Lower learning rate for stability
# train_config_m.max_iters = 30000     # Reduce if convergence is fast
# train_config_m.num_workers = 4       # Enable parallel data loading
# train_config_m.batch_size = 64       # Example batch size
# train_config_m.lr_scheduler = "cosine"  # Optionally use a learning rate scheduler

trainer_m = Trainer(train_config_m, model_m, train_dataset_m)

running on device cuda


In [7]:
train_config_ml = Trainer.get_default_config()
train_config_ml.learning_rate = 5e-3 # the model we're using is so small that we can go a bit faster
train_config_ml.max_iters = 10000
train_config_ml.num_workers = 0

# train_config_m.learning_rate = 1e-3  # Lower learning rate for stability
# train_config_m.max_iters = 30000     # Reduce if convergence is fast
# train_config_m.num_workers = 4       # Enable parallel data loading
# train_config_m.batch_size = 64       # Example batch size
# train_config_m.lr_scheduler = "cosine"  # Optionally use a learning rate scheduler

trainer_ml = Trainer(train_config_ml, model_ml, train_dataset_ml)

running on device cuda


In [8]:
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
# trainer.set_callback('on_batch_end', batch_end_callback)


In [19]:
# trainer.run() # addition

In [20]:
trainer_m.set_callback('on_batch_end', batch_end_callback)
trainer_m.run() # multiplication

iter_dt 0.00ms; iter 0: train loss 2.32628
iter_dt 33.12ms; iter 100: train loss 2.10820
iter_dt 31.25ms; iter 200: train loss 1.96571
iter_dt 38.08ms; iter 300: train loss 1.86615
iter_dt 16.02ms; iter 400: train loss 1.87027
iter_dt 16.22ms; iter 500: train loss 1.88420
iter_dt 35.62ms; iter 600: train loss 1.88278
iter_dt 15.63ms; iter 700: train loss 1.80181
iter_dt 15.63ms; iter 800: train loss 1.81552
iter_dt 32.05ms; iter 900: train loss 1.81211
iter_dt 35.59ms; iter 1000: train loss 1.84404
iter_dt 34.64ms; iter 1100: train loss 1.82641
iter_dt 33.40ms; iter 1200: train loss 1.80268
iter_dt 38.78ms; iter 1300: train loss 1.79601
iter_dt 16.89ms; iter 1400: train loss 1.79115
iter_dt 16.47ms; iter 1500: train loss 1.76678
iter_dt 22.34ms; iter 1600: train loss 1.81437
iter_dt 34.09ms; iter 1700: train loss 1.73341
iter_dt 13.74ms; iter 1800: train loss 1.75934
iter_dt 21.20ms; iter 1900: train loss 1.70473
iter_dt 30.97ms; iter 2000: train loss 1.68016
iter_dt 36.87ms; iter 2100

In [11]:
trainer_ml.set_callback('on_batch_end', batch_end_callback)
trainer_ml.run() # multiplication long

iter_dt 0.00ms; iter 0: train loss 2.32190
iter_dt 21.09ms; iter 100: train loss 1.92745
iter_dt 31.28ms; iter 200: train loss 1.78660
iter_dt 32.98ms; iter 300: train loss 1.83060
iter_dt 31.30ms; iter 400: train loss 1.74716
iter_dt 31.28ms; iter 500: train loss 1.70948
iter_dt 28.71ms; iter 600: train loss 1.69357
iter_dt 31.25ms; iter 700: train loss 1.66088
iter_dt 23.32ms; iter 800: train loss 1.64441
iter_dt 34.15ms; iter 900: train loss 1.64596
iter_dt 31.30ms; iter 1000: train loss 1.61070
iter_dt 33.07ms; iter 1100: train loss 1.66190
iter_dt 29.70ms; iter 1200: train loss 1.63283
iter_dt 31.43ms; iter 1300: train loss 1.59170
iter_dt 24.18ms; iter 1400: train loss 1.58355
iter_dt 15.45ms; iter 1500: train loss 1.60835
iter_dt 33.39ms; iter 1600: train loss 1.60153
iter_dt 31.90ms; iter 1700: train loss 1.55230
iter_dt 33.25ms; iter 1800: train loss 1.55792
iter_dt 33.92ms; iter 1900: train loss 1.49631
iter_dt 31.47ms; iter 2000: train loss 1.52181
iter_dt 39.21ms; iter 2100

In [22]:
# now let's perform some evaluation
# model.eval()
model_m.eval()
model_ml.eval()
None

In [9]:
def eval_add_split(trainer, split, max_batches, model, train_dataset, test_dataset):
    dataset = {'train':train_dataset, 'test':test_dataset}[split]
    n = train_dataset.length # naugy direct access shrug
    results = []
    mistakes_printed_already = 0
    loader = DataLoader(dataset, batch_size=100, num_workers=0, drop_last=False)
    #loader = DataLoader(dataset, batch_size=1, num_workers=0, drop_last=False)
    for b, (x, y) in enumerate(loader):
        x = x.to(trainer.device)
        y = y.to(trainer.device)

        inp = x[:, :2*n]
        sol = y[:, -n-1:]
        
        cat = model.generate(inp, n+1, do_sample=False) # using greedy argmax, not sampling
        sol_candidate = cat[:, -n-1:]         
        correct = (sol == sol_candidate).all(1).cpu() 
        for i in range(x.size(0)):
            results.append(int(correct[i]))
    
    rt = torch.tensor(results, dtype=torch.float)
    print("%s final score: %d/%d = %.2f%% correct" % (split, rt.sum(), len(results), 100*rt.mean()))
    return rt.sum()

# run a lot of examples from both train and test through the model and verify the output correctness


In [24]:
# with torch.no_grad():
#     train_score = eval_add_split(trainer, 'train', max_batches=50, model=model, train_dataset=train_dataset, test_dataset=test_dataset)
#     test_score  = eval_add_split(trainer, 'test',  max_batches=50, model=model, train_dataset=train_dataset, test_dataset=test_dataset)

In [25]:
with torch.no_grad():
    train_score = eval_add_split(trainer_m, 'train', max_batches=50, model=model_m, train_dataset=train_dataset_m, test_dataset=test_dataset_m)
    test_score  = eval_add_split(trainer_m, 'test',  max_batches=50, model=model_m, train_dataset=train_dataset_m, test_dataset=test_dataset_m)

train final score: 24/10000 = 0.24% correct
test final score: 22/10000 = 0.22% correct


In [14]:
with torch.no_grad():
    train_score = eval_add_split(trainer_ml, 'train', max_batches=50, model=model_ml, train_dataset=train_dataset_ml, test_dataset=test_dataset_ml)
    test_score  = eval_add_split(trainer_ml, 'test',  max_batches=50, model=model_ml, train_dataset=train_dataset_ml, test_dataset=test_dataset_ml)

train final score: 22/10000 = 0.22% correct
test final score: 19/10000 = 0.19% correct


In [21]:
model_config_m2 = GPT.get_default_config()
model_config_m2.model_type = 'gpt-micro'
# model_config_m.model_type = 'gpt-nano'

model_config_m2.vocab_size = train_dataset_m.get_vocab_size()
model_config_m2.block_size = train_dataset_m.get_block_size()
model_m2 = GPT(model_config_m2)


train_config_m2 = Trainer.get_default_config()
# train_config_m2.learning_rate = 5e-3 # the model we're using is so small that we can go a bit faster
# train_config_m2.max_iters = 10000
train_config_m2.num_workers = 0

train_config_m2.learning_rate = 1e-3  # Lower learning rate for stability
train_config_m2.max_iters = 100000     # Reduce if convergence is fast
train_config_m2.batch_size = 32       # Example batch size
train_config_m2.lr_scheduler = "cosine"  # Optionally use a learning rate scheduler

trainer_m2 = Trainer(train_config_m2, model_m2, train_dataset_m)

number of parameters: 0.80M
running on device cuda


In [22]:
trainer_m2.set_callback('on_batch_end', batch_end_callback)
trainer_m2.run() # multiplication m2

iter_dt 0.00ms; iter 0: train loss 2.33048
iter_dt 16.49ms; iter 100: train loss 1.90547
iter_dt 16.83ms; iter 200: train loss 1.77908
iter_dt 11.13ms; iter 300: train loss 1.69760
iter_dt 13.04ms; iter 400: train loss 1.78029
iter_dt 15.87ms; iter 500: train loss 1.68585
iter_dt 26.70ms; iter 600: train loss 1.69503
iter_dt 15.81ms; iter 700: train loss 1.71825
iter_dt 24.41ms; iter 800: train loss 1.58583
iter_dt 35.79ms; iter 900: train loss 1.57423
iter_dt 26.92ms; iter 1000: train loss 1.63410
iter_dt 31.29ms; iter 1100: train loss 1.62565
iter_dt 16.90ms; iter 1200: train loss 1.60652
iter_dt 33.23ms; iter 1300: train loss 1.63496
iter_dt 15.62ms; iter 1400: train loss 1.54459
iter_dt 24.17ms; iter 1500: train loss 1.61671
iter_dt 16.26ms; iter 1600: train loss 1.57568
iter_dt 19.67ms; iter 1700: train loss 1.51776
iter_dt 24.62ms; iter 1800: train loss 1.54051
iter_dt 22.13ms; iter 1900: train loss 1.53906
iter_dt 15.64ms; iter 2000: train loss 1.51222
iter_dt 15.67ms; iter 2100

In [23]:
model_m2.eval()
with torch.no_grad():
    train_score_m2 = eval_add_split(trainer_m2, 'train', max_batches=50, model=model_m2, train_dataset=train_dataset_m, test_dataset=test_dataset_m)
    test_score_m2  = eval_add_split(trainer_m2, 'test',  max_batches=50, model=model_m2, train_dataset=train_dataset_m, test_dataset=test_dataset_m)

train final score: 23/10000 = 0.23% correct
test final score: 26/10000 = 0.26% correct
